# BayeSN SNIa

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from lightcurvelynx.astro_utils.passbands import PassbandGroup
from lightcurvelynx.astro_utils.pzflow_node import PZFlowNode
from lightcurvelynx.astro_utils.snia_utils import (
    DistModFromRedshift,
    HostmassX1Func,
    X0FromDistMod,
)
from lightcurvelynx.math_nodes.np_random import NumpyRandomFunc
from lightcurvelynx.obstable.opsim import OpSim
from lightcurvelynx.simulate import simulate_lightcurves
from lightcurvelynx.models.sncomso_models import SncosmoWrapperModel
from lightcurvelynx.models.snia_host import SNIaHost
from lightcurvelynx.utils.plotting import plot_lightcurves

from lightcurvelynx import _LIGHTCURVELYNX_BASE_DATA_DIR

2025-07-24 16:51:37,701 - INFO - Note: NumExpr detected 10 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2025-07-24 16:51:37,703 - INFO - NumExpr defaulting to 8 threads.


## Load Data Files

We start by loading the files we will need for running the simulation: the OpSim database and the passband information. Both of these live in the `data/` directory in the root directory. Note that nothing in this directory is saved to github, so the files might have to be downloaded initially.

In [ ]:
# Load the OpSim data.
opsim_db = OpSim.from_db(_LIGHTCURVELYNX_BASE_DATA_DIR / "baseline_v3.4_10yrs.db")
t_min, t_max = opsim_db.time_bounds()
print(f"Loaded OpSim with {len(opsim_db)} rows and times [{t_min}, {t_max}]")

# Load the passband data for the griz filters only.
# Use a (possibly older) cached version of the passbands to avoid downloading them.
table_dir = "../../tests/lightcurvelynx/data/passbands"
passband_group = PassbandGroup(
    preset="LSST",
    filters_to_load=["g", "r", "i", "z"],
    units="nm",
    trim_quantile=0.001,
    delta_wave=1,
    table_dir=table_dir,
)
# passband_group = PassbandGroup.from_preset(preset="LSST", table_dir=table_dir)
print(f"Loaded Passbands: {passband_group}")

In [ ]:
# Create a mask of matching filters.
filter_mask = passband_group.mask_by_filter(opsim_db["filter"])

# Filter the OpSim
opsim_db = opsim_db.filter_rows(filter_mask)
t_min, t_max = opsim_db.time_bounds()
print(f"Filtered OpSim to {len(opsim_db)} rows and times [{t_min}, {t_max}]")

## Create the model

To generate simulationed light curves we need to define the proporties of the object from which to sample. We start by creating a host based on a pre-trained pzflow model.

In [ ]:
# Load the Flow model into a PZFlow node. This gives access to all of the outputs of the
# flow model as attributes of the PZFlowNode.
pz_node = PZFlowNode.from_file(
    _LIGHTCURVELYNX_BASE_DATA_DIR / "model_files" / "snia_hosts_test_pzflow.pkl",  # filename
    node_label="pznode",
)

# Create a model for the host of the SNIa. The attributes will be sampled via
# the PZFlowNode's model. So each host instantiation will have its own properties.
# Note: This requires the user to know the output names from the underlying flow model.
host = SNIaHost(
    ra=pz_node.RA_GAL,
    dec=pz_node.DEC_GAL,
    hostmass=pz_node.LOGMASS,
    redshift=NumpyRandomFunc("uniform", low=0.1, high=0.6),
    node_label="host",
)

Here we define the Amplitude class with the purpose to create the distance modulus normalization factor to be added as a parameter to the bayesn model.

In [ ]:
from lightcurvelynx.base_models import FunctionNode


class AmplitudeFromDistMod(FunctionNode):
    def __init__(self, distmod, m_abs, **kwargs):
        # Call the super class's constructor with the needed information.
        super().__init__(
            func=self._amplitude_given_distmod,
            distmod=distmod,
            m_abs=m_abs,
            **kwargs,
        )

    def _amplitude_given_distmod(self, distmod, m_abs):
        """Compute distance modulus given redshift and cosmology.

        Parameters
        ----------
        distmod: float or numpy.ndarray
            The distance modulus (in mag)
        m_abs: float or numpy.ndarry
            The absolute magnitude modulus (in mag)

        Returns
        -------
        amplitude : float or numpy.ndarray
            The distance modulus effect
        """
        return np.power(10.0, -0.4 * (distmod + m_abs))

Next we create the SNIa model itself. We use bayeSN model with parameters randomly generated from realistic distributions.

Note that some attributes, such as (RA, dec), are sampled relative to the host's properties.

In [ ]:
bayesian_model_name = "bayesnModel"
distmod_func = DistModFromRedshift(host.redshift, H0=73.0, Omega_m=0.3)
m_abs_func = NumpyRandomFunc("normal", loc=-19.3, scale=0.1)
amplitude_func = AmplitudeFromDistMod(distmod_func, m_abs_func)
source = BayesnModel(
    theta=NumpyRandomFunc("uniform", low=-1.74, high=2.10),
    Av=NumpyRandomFunc("uniform", low=0.01, high=0.3),
    Rv=NumpyRandomFunc("uniform", low=3, high=3),
    t0=NumpyRandomFunc("uniform", low=t_min, high=t_max),
    ra=NumpyRandomFunc("normal", loc=host.ra, scale=0.01),
    dec=NumpyRandomFunc("normal", loc=host.dec, scale=0.01),
    redshift=host.redshift,
    node_label="source",
    Amplitude=amplitude_func,
)

## Generate the simulations

We can now generate random simulations with all the information defined above. The light curves are written in the [nested-pandas](https://github.com/lincc-frameworks/nested-pandas) format for easy analysis. 

In [ ]:
lightcurves = simulate_lightcurves(source, 1_000, opsim_db, passband_group)
print(lightcurves)

Now let's plot some random light curves

In [ ]:
random_ids = np.random.choice(len(lightcurves), 10)

for random_id in random_ids:
    # Extract the row for this object.
    lc = lightcurves.loc[random_id]

    if lc["nobs"] > 0:
        # Unpack the nested columns (filters, mjd, flux, and flux error).
        lc_filters = np.asarray(lc["lightcurve"]["filter"], dtype=str)
        lc_mjd = np.asarray(lc["lightcurve"]["mjd"], dtype=float)
        lc_flux = np.asarray(lc["lightcurve"]["flux"], dtype=float)
        lc_fluxerr = np.asarray(lc["lightcurve"]["fluxerr"], dtype=float)

        plot_lightcurves(
            fluxes=lc_flux,
            times=lc_mjd,
            fluxerrs=lc_fluxerr,
            filters=lc_filters,
        )

We can also plot the light curves in magnitude

In [ ]:
lc_mag = -2.5 * np.log10(lc_flux) + 31.4
lc_magerr = np.absolute(1.086 * lc_fluxerr / lc_flux)

plot_lightcurves(
    fluxes=lc_mag,
    times=lc_mjd,
    fluxerrs=lc_magerr,
    filters=lc_filters,
)
plt.title(lc["z"])
plt.ylabel("Mag")
plt.ylim(plt.ylim()[::-1])